# AutoMPC Demo

Welcome!  This notebook demonstrates the core features of AutoMPC.  We will use the Cart-Pole swing-up task as an example, and we will tune an MPC pipeline consisting of a multi-layer perception (MLP) system ID model and an iLQR optimizer.

## Set-Up

To begin, we need an input dataset and specifications for the system and task.  The `benchmarks` module provides these for a few example systems.

In [1]:
import os
os.chdir("..")
import autompc as ampc
import numpy as np
from autompc.benchmarks import CartpoleSwingupBenchmark

benchmark = CartpoleSwingupBenchmark()


# Get system and task specification
system = benchmark.system
task   = benchmark.task

# Generate benchmark dataset
trajs = benchmark.gen_trajs(seed=100, n_trajs=500, traj_len=200)

Cannot import SnoptWrapper
Cannot import KnitroSolver, make sure its Python interface is installed


Next, we need to declare our MPC pipeline.  The following code initializes a pipeline with a MLP system ID model, a quadratic objective function, and and an iLQR optimizer.

In [2]:
from autompc.sysid import MLPFactory
from autompc.control import IterativeLQRFactory
from autompc.costs import QuadCostFactory

model_factory = MLPFactory(system)
ctrlr_factory = IterativeLQRFactory(system)
cost_factory  = QuadCostFactory(system)

pipeline = ampc.Pipeline(system, model_factory, ctrlr_factory, cost_factory)

We can view the joint configuration space of the pipeline

In [3]:
pipeline.get_configuration_space()

Configuration space object:
  Hyperparameters:
    _cost:dx_F, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:dx_Q, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:omega_F, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:omega_Q, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:theta_F, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:theta_Q, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:u_R, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:x_F, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:x_Q, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _ctrlr:horizon, Type: UniformInteger, Range: [5, 25], Default: 20
    _model:hidden_size_1, Type: UniformInteger, Range: [16, 256], Default: 128
   

## Tuning

Tune with selected surrogate model class, default model configuration.

In [4]:
from autompc.tuning import PipelineTuner

surrogate_factory = MLPFactory(system)
tuner = PipelineTuner(surrogate_mode="defaultcfg", surrogate_factory=surrogate_factory, surrogate_split=0.5)

controller, tune_result = tuner.run(pipeline, task, trajs, n_iters=100, rng=np.random.default_rng(100))

/usr/lib/python3/dist-packages/pyparsing.py:1745: FutureWarning: Possible set intersection at position 3
  self.re = re.compile( self.reString )
/home/william/.local/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


use_cuda=True
MLP Using Cuda
hidden_sizes= [128, 128]
100%|██████████| 50/50 [01:19<00:00,  1.59s/it]

INFO:smac.utils.io.cmd_reader.CMDReader:Output to smac3-output_2021-05-02_13:00:45_871237
INFO:smac.facade.smac_hpo_facade.SMAC4HPO:Optimizing a deterministic scenario for quality without a tuner timeout - will make SMAC deterministic and only evaluate one configuration per iteration!
INFO:smac.initial_design.sobol_design.SobolDesign:Running initial design for 25 configurations
INFO:smac.facade.smac_hpo_facade.SMAC4HPO:<class 'smac.facade.smac_hpo_facade.SMAC4HPO'>
INFO:smac.optimizer.smbo.SMBO:Running initial design
INFO:smac.intensification.intensification.Intensifier:First run, no incumbent provided; challenger is assumed to be the incumbent



use_cuda=True
MLP Using Cuda
hidden_sizes= [136, 136, 136]
100%|██████████| 50/50 [01:41<00:00,  2.03s/it]
Simulating Surrogate Trajectory: 
 99%|█████████▉| 198/200 [00:10<00:00, 18.54it/s]

INFO:smac.intensification.intensification.Intensifier:First run, no incumbent provided; challenger is assumed to be the incumbent


...


use_cuda=True
MLP Using Cuda
hidden_sizes= [145, 130]
100%|██████████| 50/50 [01:21<00:00,  1.63s/it]
Simulating Surrogate Trajectory: 
 99%|█████████▉| 198/200 [00:20<00:00,  9.57it/s]

INFO:smac.intensification.intensification.Intensifier:Wallclock time limit for intensification reached (used: 102.443050 sec, available: 0.000010 sec)



use_cuda=True
MLP Using Cuda
hidden_sizes= [191, 172, 125, 109]
100%|██████████| 50/50 [02:03<00:00,  2.47s/it]
Simulating Surrogate Trajectory: 
 99%|█████████▉| 198/200 [01:04<00:00,  3.07it/s]

INFO:smac.intensification.intensification.Intensifier:Wallclock time limit for intensification reached (used: 187.840149 sec, available: 0.000010 sec)
INFO:smac.stats.stats.Stats:##########################################################
INFO:smac.stats.stats.Stats:Statistics:
INFO:smac.stats.stats.Stats:#Incumbent changed: 9
INFO:smac.stats.stats.Stats:#Submitted target algorithm runs: 100 / 100.0
INFO:smac.stats.stats.Stats:#Finished target algorithm runs: 100 / 100.0
INFO:smac.stats.stats.Stats:#Configurations: 100
INFO:smac.stats.stats.Stats:Used wallclock time: 13443.35 / inf sec 
INFO:smac.stats.stats.Stats:Used target algorithm runtime: 13326.83 / inf sec
INFO:smac.stats.stats.Stats:##########################################################
INFO:smac.facade.smac_hpo_facade.SMAC4HPO:Final Incumbent: Configuration:
  _cost:dx_F, Value: 0.0025735397668133608
  _cost:dx_Q, Value: 0.6203655997769277
  _cost:omega_F, Value: 8359.436202123476
  _cost:omega_Q, Value: 23.714515176435285



use_cuda=True
MLP Using Cuda
hidden_sizes= [199, 205, 132, 144]
100%|██████████| 50/50 [04:08<00:00,  4.96s/it]


We can also set the configuration of the surrogate model.

In [7]:
surrogate_cfg = surrogate_factory.get_configuration_space().get_default_configuration()
surrogate_cfg["n_hidden_layers"] = "2"
surrogate_cfg["hidden_size_1"] = 128
surrogate_cfg["hidden_size_2"] = 128       
surrogate_cfg["nonlintype"] = "relu"

tuner = PipelineTuner(surrogate_mode="fixedcfg", surrogate_factory=surrogate_factory,
                     surrogate_cfg=surrogate_cfg, surrogate_split=0.5)

controller, tune_result2 = tuner.run(pipeline, task, trajs, n_iters=100, rng=np.random.default_rng(100),
                                    truedyn=benchmark.dynamics)

use_cuda=True
MLP Using Cuda
hidden_sizes= [128, 128]
100%|██████████| 50/50 [01:22<00:00,  1.66s/it]

INFO:smac.utils.io.cmd_reader.CMDReader:Output to smac3-output_2021-05-02_18:21:52_194294
INFO:smac.facade.smac_hpo_facade.SMAC4HPO:Optimizing a deterministic scenario for quality without a tuner timeout - will make SMAC deterministic and only evaluate one configuration per iteration!
INFO:smac.initial_design.sobol_design.SobolDesign:Running initial design for 25 configurations
INFO:smac.facade.smac_hpo_facade.SMAC4HPO:<class 'smac.facade.smac_hpo_facade.SMAC4HPO'>
INFO:smac.optimizer.smbo.SMBO:Running initial design
INFO:smac.intensification.intensification.Intensifier:First run, no incumbent provided; challenger is assumed to be the incumbent



use_cuda=True
MLP Using Cuda
hidden_sizes= [136, 136, 136]
100%|██████████| 50/50 [01:42<00:00,  2.04s/it]
Simulating Surrogate Trajectory: 
 99%|█████████▉| 198/200 [00:09<00:00, 19.83it/s]
Simulating True Dynamics Trajectory
 99%|█████████▉| 198/200 [00:10<00:00, 19.67it/s]

INFO:smac.intensification.intensification.Intensifier:First run, no incumbent provided; challenger is assumed to be the incumbent
INFO:smac.intensification.intensification.Intensifier:Updated estimated cost of incumbent on 1 runs: 200.0000



use_cuda=True
MLP Using Cuda
hidden_sizes= [76, 76]
100%|██████████| 50/50 [01:20<00:00,  1.61s/it]
Simulating Surrogate Trajectory: 
  4%|▍         | 9/200 [00:00<00:04, 40.37it/s]

...


use_cuda=True
MLP Using Cuda
hidden_sizes= [42, 56]
100%|██████████| 50/50 [01:21<00:00,  1.64s/it]
Simulating Surrogate Trajectory: 
 99%|█████████▉| 198/200 [00:06<00:00, 28.78it/s]
Simulating True Dynamics Trajectory
 99%|█████████▉| 198/200 [00:06<00:00, 29.62it/s]

INFO:smac.intensification.intensification.Intensifier:Wallclock time limit for intensification reached (used: 95.464433 sec, available: 0.000010 sec)



use_cuda=True
MLP Using Cuda
hidden_sizes= [174, 199]
100%|██████████| 50/50 [01:23<00:00,  1.67s/it]
Simulating Surrogate Trajectory: 
 99%|█████████▉| 198/200 [00:21<00:00,  9.12it/s]
Simulating True Dynamics Trajectory
 99%|█████████▉| 198/200 [00:20<00:00,  9.44it/s]

INFO:smac.intensification.intensification.Intensifier:Wallclock time limit for intensification reached (used: 126.049941 sec, available: 0.000010 sec)
INFO:smac.stats.stats.Stats:##########################################################
INFO:smac.stats.stats.Stats:Statistics:
INFO:smac.stats.stats.Stats:#Incumbent changed: 5
INFO:smac.stats.stats.Stats:#Submitted target algorithm runs: 100 / 100.0
INFO:smac.stats.stats.Stats:#Finished target algorithm runs: 100 / 100.0
INFO:smac.stats.stats.Stats:#Configurations: 100
INFO:smac.stats.stats.Stats:Used wallclock time: 19838.05 / inf sec 
INFO:smac.stats.stats.Stats:Used target algorithm runtime: 19708.78 / inf sec
INFO:smac.stats.stats.Stats:##########################################################
INFO:smac.facade.smac_hpo_facade.SMAC4HPO:Final Incumbent: Configuration:
  _cost:dx_F, Value: 0.1539926526059493
  _cost:dx_Q, Value: 0.017673128603665555
  _cost:omega_F, Value: 3651.7412725483823
  _cost:omega_Q, Value: 19.91103049494464
 


use_cuda=True
MLP Using Cuda
hidden_sizes= [211, 156, 170, 133]
100%|██████████| 50/50 [04:09<00:00,  4.99s/it]


We can also auto-tune a particular surrogate class

In [8]:
tuner = PipelineTuner(surrogate_mode="autotune", surrogate_factory=surrogate_factory, surrogate_split=0.5)

controller, tune_result3 = tuner.run(pipeline, task, trajs, n_iters=100, rng=np.random.default_rng(100),
                                    truedyn=benchmarks.dynamics)

/home/william/.local/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
INFO:smac.utils.io.cmd_reader.CMDReader:Output to smac3-output_2021-05-02_23:56:56_364282
INFO:smac.facade.smac_hpo_facade.SMAC4HPO:Optimizing a deterministic scenario for quality without a tuner timeout - will make SMAC deterministic and only evaluate one configuration per iteration!
INFO:smac.initial_design.sobol_design.SobolDesign:Running initial design for 25 configurations
INFO:smac.facade.smac_hpo_facade.SMAC4HPO:<class 'smac.facade.smac_hpo_facade.SMAC4HPO'>
INFO:smac.optimizer.smbo.SMBO:Running initial design
INFO:smac.intensification.intensification.Intensifier:First run, no incumbent provided

Evaluating cfg:
Configuration:
  _MLP:hidden_size_1, Value: 136
  _MLP:hidden_size_2, Value: 136
  _MLP:hidden_size_3, Value: 136
  _MLP:lr, Value: 0.003162277660168379
  _MLP:n_hidden_layers, Value: '3'
  _MLP:nonlintype, Value: 'sigmoid'
  model, Value: 'MLP'

use_cuda=True
MLP Using Cuda
hidden_sizes= [136, 136, 136]
100%|██████████| 50/50 [01:11<00:00,  1.44s/it]

INFO:smac.intensification.intensification.Intensifier:First run, no incumbent provided; challenger is assumed to be the incumbent
INFO:smac.intensification.intensification.Intensifier:Updated estimated cost of incumbent on 1 runs: 0.0428



Evaluating cfg:
Configuration:
  _MLP:hidden_size_1, Value: 76
  _MLP:hidden_size_2, Value: 76
  _MLP:lr, Value: 0.056234132519034884
  _MLP:n_hidden_layers, Value: '2'
  _MLP:nonlintype, Value: 'selu'
  model, Value: 'MLP'

use_cuda=True
MLP Using Cuda
hidden_sizes= [76, 76]
100%|██████████| 50/50 [01:02<00:00,  1.25s/it]

INFO:smac.intensification.intensification.Intensifier:Wallclock time limit for intensification reached (used: 134.676168 sec, available: 0.000010 sec)


...


Evaluating cfg:
Configuration:
  _MLP:hidden_size_1, Value: 121
  _MLP:hidden_size_2, Value: 151
  _MLP:hidden_size_3, Value: 241
  _MLP:hidden_size_4, Value: 31
  _MLP:lr, Value: 8.659643233600651e-05
  _MLP:n_hidden_layers, Value: '4'
  _MLP:nonlintype, Value: 'sigmoid'
  model, Value: 'MLP'

use_cuda=True
MLP Using Cuda
hidden_sizes= [121, 151, 241, 31]
100%|██████████| 50/50 [01:32<00:00,  1.84s/it]

INFO:smac.intensification.intensification.Intensifier:Wallclock time limit for intensification reached (used: 92.229549 sec, available: 0.000010 sec)



Evaluating cfg:
Configuration:
  _MLP:hidden_size_1, Value: 241
  _MLP:hidden_size_2, Value: 31
  _MLP:lr, Value: 0.027384196342643607
  _MLP:n_hidden_layers, Value: '2'
  _MLP:nonlintype, Value: 'relu'
  model, Value: 'MLP'

use_cuda=True
MLP Using Cuda
hidden_sizes= [241, 31]
 18%|█▊        | 9/50 [00:11<00:51,  1.26s/it]

INFO:smac.stats.stats.Stats:##########################################################
INFO:smac.stats.stats.Stats:Statistics:
INFO:smac.stats.stats.Stats:#Incumbent changed: 0
INFO:smac.stats.stats.Stats:#Submitted target algorithm runs: 14 / 100.0
INFO:smac.stats.stats.Stats:#Finished target algorithm runs: 14 / 100.0
INFO:smac.stats.stats.Stats:#Configurations: 14
INFO:smac.stats.stats.Stats:Used wallclock time: 998.79 / inf sec 
INFO:smac.stats.stats.Stats:Used target algorithm runtime: 987.35 / inf sec
INFO:smac.stats.stats.Stats:##########################################################
INFO:smac.facade.smac_hpo_facade.SMAC4HPO:Final Incumbent: Configuration:
  _MLP:hidden_size_1, Value: 136
  _MLP:hidden_size_2, Value: 136
  _MLP:hidden_size_3, Value: 136
  _MLP:lr, Value: 0.003162277660168379
  _MLP:n_hidden_layers, Value: '3'
  _MLP:nonlintype, Value: 'sigmoid'
  model, Value: 'MLP'

INFO:smac.facade.smac_hpo_facade.SMAC4HPO:Estimated cost of incumbent: 0.042791


KeyboardInterrupt: 

Finally, we can auto-select and auto-tune the surrogate mode class

In [ ]:
tuner = PipelineTuner(surrogate_mode="autoselect", surrogate_factory=surrogate_factory, surrogate_split=0.5)

controller, tune_result4 = tuner.run(pipeline, task, trajs, n_iters=100, rng=np.random.default_rng(100),
                                    truedyn=benchmark.dynamics)

Finally, we can pre-train and select our own surrogate.

In [ ]:
from autompc.tuning import ModelTuner
from autompc.evaluation import HoldoutModelEvaluator

sysid_trajs = trajs[:500]
surr_trajs = trajs[500:]

model_evaluator = HoldoutModelEvaluator(system=system,trajs=surr_trajs,rng=np.random.default_rng(100),
                                        holdout_prop=0.25, metric="rmse")
model_tuner = ModelTuner(system, model_evaluator)
model_tuner.add_model_factory(surrogate_factory)

surrogate_model, _ = model_tuner.run(rng=np.random.default_rng(100), n_iters = 1)

tuner = PipelineTuner(surrogate_mode="pretrain")

controller, tune_result5 = tuner.run(pipeline, task, sysid_trajs, n_iters=100, surrogate=surrogate_model
                                    truedyn=benchmark.dynamics)

## Decoupled Tuning

The above examples is full pipeline tuning, which searches the configuration space of all components simultaneously.  Alternatively, we can take a decoupled tuning approach, where the model is first tuned based on prediction accuracy, then the objective function and optimizer are tuned.

First, we must tune the model

In [ ]:
from autompc.tuning import ModelTuner
from autompc.evaluation import HoldoutModelEvaluator

model_evaluator = HoldoutModelEvaluator(holdout_prop=0.25, metric="rmse")
model_tuner = ModelTuner(model_evaluator)

model, model_tune_result = model_tuner.run(model_factory, trajs, n_iters=100)

Now that we have tuned the model, we can create a pipeline with the pre-tuned model, and then run pipeline tuning.

In [ ]:
pipeline_fixed_model = ampc.Pipeline(system, model, iLQRFactory, QuadCostFactory)
controller6, tune_result6 = tuner.run(pipeline, task, trajs, n_iters=100, truedyn=benchmark.dynamics)   